In [34]:
#读入数据
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from matplotlib import pyplot as plt

trainData = pd.read_csv("./nlp-getting-started-data/train.csv")
trainData['text'] = trainData['text'].fillna('')

trainKeyword=trainData.values[:,1]
trainLocation=trainData.values[:,2]
trainText=trainData.values[:,3]
trainTarget=trainData.values[:,4]

testData = pd.read_csv("./nlp-getting-started-data/test.csv")
testData['text'] = testData['text'].fillna('')

testKeyword=testData.values[:,1]
testLocation=testData.values[:,2]
testText=testData.values[:,3]

In [35]:
'''
import seaborn as sns #一个Matplotlib高级数据可视化库，画统计图表

trainData['targetMean']=trainData.groupby('keyword')['target'].transform('mean')
fig = plt.figure(figsize=(8,27))
sns.countplot(y=trainData.sort_values(by='targetMean',ascending=False)['keyword'], hue=trainData.sort_values(by='targetMean',ascending=False)['target']) #按照targetMean排序，默认升序，ascending=False改为降序排序

plt.legend(loc=1)

plt.show()

trainData.drop(columns=['targetMean'], inplace=True) #inplace=True的时候在原来的数据上发生改变，默认为inplace=False不改变原数据
'''

"\nimport seaborn as sns #一个Matplotlib高级数据可视化库，画统计图表\n\ntrainData['targetMean']=trainData.groupby('keyword')['target'].transform('mean')\nfig = plt.figure(figsize=(8,27))\nsns.countplot(y=trainData.sort_values(by='targetMean',ascending=False)['keyword'], hue=trainData.sort_values(by='targetMean',ascending=False)['target']) #按照targetMean排序，默认升序，ascending=False改为降序排序\n\nplt.legend(loc=1)\n\nplt.show()\n\ntrainData.drop(columns=['targetMean'], inplace=True) #inplace=True的时候在原来的数据上发生改变，默认为inplace=False不改变原数据\n"

In [36]:
#Tokenizer分词器
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    cache_dir='./myModels', #下载的模型储存的位置
)


In [ ]:
from torch.utils.data import Dataset,DataLoader
class TweetDataset(Dataset):
    def __init__(self, texts, labels=None, max_len=256): #初始化。
        self.texts=texts
        self.labels=labels
        self.max_len=max_len
    def __len__(self): #获取包含的样本数
        return len(self.texts)
    def __getitem__(self, idx): #返回索引idx处的数据集样本
        text = str(self.texts[idx])
        encoding = tokenizer.encode_plus( #用于文本编码，会返回：1.input_ids:token ID序列(包括[CLS](通常表示句子或文档开头，对应输入文本第一个词向量)和[SEP](对应最后一个词向量，分割不同句子，句子间用来表示分界点)) 2.token_type_ids句子区分标记 3.attention_mask指示有效token(为1)和无效token(为0)
            text, #文本
            add_special_tokens=True, #是否添加[CLS]和[SEP]
            max_length=self.max_len, #最大序列长度
            padding='max_length', #是否填充
            truncation=True, #是否截断
            return_attention_mask=True, #是否返回attention_mask
            return_tensors='pt', #返回tf(Tensorflow的Tensor)、pt(PyTorch的)或np(Numpy的ndarray)
        )
        input_ids = encoding['input_ids'].flatten() #.flatten()用于将多维数据降为一维数据
        attention_mask = encoding['attention_mask'].flatten()
        
        if len(input_ids) > self.max_len: #不知道为啥虽然写了padding='max_length'但还是会有没填充到max_len的情况，需要手动搞。
            input_ids = input_ids[:self.max_len]
            attention_mask = attention_mask[:self.max_len]
        elif len(input_ids) < self.max_len:
            # 手动填充
            pad_length = self.max_len - len(input_ids)
            input_ids = torch.cat([
                input_ids, 
                torch.full((pad_length,), tokenizer.pad_token_id, dtype=torch.long)
            ])
            attention_mask = torch.cat([
                attention_mask, 
                torch.zeros(pad_length, dtype=torch.long)
            ])
        
        # 添加长度检查
        assert len(input_ids) == self.max_len, f"input_ids length {len(input_ids)} != {self.max_len}"
        assert len(attention_mask) == self.max_len, f"attention_mask length {len(attention_mask)} != {self.max_len}"
        
        if self.labels is not None: #如果有labels则是训练集
            labels = torch.tensor(self.labels[idx], dtype=torch.long)
            return {
                'input_ids':input_ids,
                'attention_mask':attention_mask,
                'labels':labels
            }
        else: #没有labels则是测试集
            return {
                'input_ids':input_ids,
                'attention_mask':attention_mask
            }
        

In [38]:
#构造数据集，训练集和验证集
from sklearn.model_selection import train_test_split
trainDataList, testDataList, trainClassList, testClassList = train_test_split(trainData['text'], trainData['target'], test_size=0.1, random_state=233) #划分训练集和验证集，由于训练的有点慢所以test_size较大先试一下

trainDataset = TweetDataset(trainDataList.tolist(), trainClassList.tolist()) #.tolist()可以将任意形状的Tensor转换为Python原生标量
#当下标为8的数据划分到testClassList的话，这里运行print(trainClassList[8])会报错，这是以你为trainClassList 是一个 pandas Series，其索引可能不是连续的，不存在索引为8的数据
testDataset = TweetDataset(testDataList.tolist(), testClassList.tolist())
BATCH_SIZE=16
trainLoader = DataLoader(trainDataset, batch_size = BATCH_SIZE, shuffle = True)
testLoader = DataLoader(testDataset, batch_size = BATCH_SIZE)


In [39]:
#加载BERT模型
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained( #BERT模型的序列分类版本
    'bert-base-uncased', #BERT基础版本
    cache_dir='./myModels', #下载的模型储存的位置,不设置默认会在用户文件夹下/.cache/torch/transformers里
    num_labels = 2, #二分类
    output_attentions = False, #不返回注意力权重矩阵
    output_hidden_states = False #不返回所有隐藏层的输出，进返回最后一层
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('当前device为' + str(device))
model=model.to(device)

当前device为cuda


In [40]:
#循环训练
from transformers import AdamW
epochs = 3 #迭代次数
optimizer = AdamW(model.parameters(), lr=2e-5)


for epoch in range(epochs):
    model.train() #训练模式
    i=0
    for batch in trainLoader:
        print('epoch is ' + str(epoch) + ' i is ' + str(i))
        i=i+1
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
    
        optimizer.zero_grad() #清空模型参数梯度
        loss = outputs[0]
        loss.backward() #计算损失函数对模型参数的梯度
        optimizer.step() #更新模型参数

epoch is 0 i is 0
epoch is 0 i is 1
epoch is 0 i is 2
epoch is 0 i is 3
epoch is 0 i is 4
epoch is 0 i is 5
epoch is 0 i is 6
epoch is 0 i is 7
epoch is 0 i is 8
epoch is 0 i is 9
epoch is 0 i is 10
epoch is 0 i is 11
epoch is 0 i is 12
epoch is 0 i is 13
epoch is 0 i is 14
epoch is 0 i is 15
epoch is 0 i is 16
epoch is 0 i is 17
epoch is 0 i is 18
epoch is 0 i is 19
epoch is 0 i is 20
epoch is 0 i is 21
epoch is 0 i is 22
epoch is 0 i is 23
epoch is 0 i is 24
epoch is 0 i is 25
epoch is 0 i is 26
epoch is 0 i is 27
epoch is 0 i is 28
epoch is 0 i is 29
epoch is 0 i is 30
epoch is 0 i is 31
epoch is 0 i is 32
epoch is 0 i is 33
epoch is 0 i is 34
epoch is 0 i is 35
epoch is 0 i is 36
epoch is 0 i is 37
epoch is 0 i is 38
epoch is 0 i is 39
epoch is 0 i is 40
epoch is 0 i is 41
epoch is 0 i is 42
epoch is 0 i is 43
epoch is 0 i is 44
epoch is 0 i is 45
epoch is 0 i is 46
epoch is 0 i is 47
epoch is 0 i is 48
epoch is 0 i is 49
epoch is 0 i is 50
epoch is 0 i is 51
epoch is 0 i is 52
epo

In [ ]:
testDataset = TweetDataset(testData['text'].tolist()) #注意这里和上面同名了(实在不会起名QAQ)
testLoader = DataLoader(testDataset, batch_size = BATCH_SIZE)

model.eval()#预测模式
predictions = []
with torch.no_grad(): #Pytorch默认跟踪张量以计算梯度，torch.no_grad()会临时禁用跟踪。且能防止意外的权重更新
    for batch in testLoader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(preds) #添加多个元素的.append()

myAns = pd.DataFrame({'id': testData['id'], 'target': predictions})
myAns.to_csv("nlp-getting-started-data/myAns.csv", index=False, sep=',') #当前版本正确率0.83879